In [12]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import json


In [13]:
import datetime as dt

In [14]:
!pip install dataprep
!pip install dask
!pip install pandas_profiling

In [15]:
import cx_Oracle
import yaml
import os
from pathlib import Path
home = str(Path.home())

def process_yaml():
	with open("../config.yaml") as file:
		return yaml.safe_load(file)


class OracleJSONDatabaseConnection:
    def __init__(self, data=process_yaml()):
        # wallet location (default is HOME/wallets/wallet_X)
        os.environ['TNS_ADMIN'] = '{}/{}'.format(home, process_yaml()['WALLET_DIR'])
        print(os.environ['TNS_ADMIN'])
        self.pool = cx_Oracle.SessionPool(data['db']['username'], data['db']['password'], data['db']['dsn'],
            min=1, max=4, increment=1, threaded=True,
            getmode=cx_Oracle.SPOOL_ATTRVAL_WAIT
        )
        print('Connection successful.')



    def close_pool(self):
        self.pool.close()
        print('Connection pool closed.')



    def insert(self, collection_name, json_object_to_insert):
        connection = self.pool.acquire()
        connection.autocommit = True
        soda = connection.getSodaDatabase()
        x_collection = soda.createCollection(collection_name)

        try:
            x_collection.insertOne(json_object_to_insert)
            print('[DBG] INSERT {} OK'.format(json_object_to_insert))
        except cx_Oracle.IntegrityError:
            print('[DBG] INSERT {} ERR'.format(json_object_to_insert))
            return 0
        self.pool.release(connection)
        return 1



    def delete(self, collection_name, on_column, on_value):
        connection = self.pool.acquire()
        soda = connection.getSodaDatabase()
        x_collection = soda.createCollection(collection_name)
        qbe = {on_column: on_value}
        x_collection.find().filter(qbe).remove()
        self.pool.release(connection)



    def get_connection(self):
        return self.pool.acquire() 



    def close_connection(self, conn_object):
        self.pool.release(conn_object)



    def get_collection_names(self):
        connection = self.pool.acquire()
        returning_object = connection.getSodaDatabase().getCollectionNames(startName=None, limit=0)
        self.pool.release(connection)
        return returning_object



    def open_collection(self, collection_name):
        connection = self.pool.acquire()
        returning_object = self.pool.acquire().getSodaDatabase().openCollection(collection_name)
        self.pool.release(connection)
        return returning_object




def test_class():
    object = OracleJSONDatabaseConnection()
    print(object.pool)
    object.close_pool()

In [44]:
def build_final_object(json_object):
	all_frames = list()   

	match_id = json_object.get('metadata').get('matchId')

	iterator = 0
	winner = int()
	# Determine winner
	frames = json_object.get('info').get('frames')
	last_frame = frames[-1]
	last_event = last_frame.get('events')[-1]
	assert last_event.get('type') == 'GAME_END'
	winner = last_event.get('winningTeam')
	print(last_event.get('winningTeam'))

	for x in json_object.get('info').get('frames'):
		frame = {
			"timestamp": x.get('timestamp')
		}		

		for y in range(1, 11):
			frame['abilityPower'] = x.get('participantFrames').get('{}'.format(y)).get('championStats').get('abilityPower')
			frame['armor'] = x.get('participantFrames').get('{}'.format(y)).get('championStats').get('armor')
			frame['armorPen'] = x.get('participantFrames').get('{}'.format(y)).get('championStats').get('armorPen')
			frame['armorPenPercent'] = x.get('participantFrames').get('{}'.format(y)).get('championStats').get('armorPenPercent')
			frame['attackDamage'] = x.get('participantFrames').get('{}'.format(y)).get('championStats').get('attackDamage')
			frame['attackSpeed'] = x.get('participantFrames').get('{}'.format(y)).get('championStats').get('attackSpeed')
			frame['bonusArmorPenPercent'] = x.get('participantFrames').get('{}'.format(y)).get('championStats').get('bonusArmorPenPercent')
			frame['bonusMagicPenPercent'] = x.get('participantFrames').get('{}'.format(y)).get('championStats').get('bonusMagicPenPercent')
			frame['ccReduction'] = x.get('participantFrames').get('{}'.format(y)).get('championStats').get('ccReduction')
			frame['cooldownReduction'] = x.get('participantFrames').get('{}'.format(y)).get('championStats').get('cooldownReduction')
			frame['health'] = x.get('participantFrames').get('{}'.format(y)).get('championStats').get('health')
			frame['healthMax'] = x.get('participantFrames').get('{}'.format(y)).get('championStats').get('healthMax')
			frame['healthRegen'] = x.get('participantFrames').get('{}'.format(y)).get('championStats').get('healthRegen')
			frame['lifesteal'] = x.get('participantFrames').get('{}'.format(y)).get('championStats').get('lifesteal')
			frame['magicPen'] = x.get('participantFrames').get('{}'.format(y)).get('championStats').get('magicPen')
			frame['magicPenPercent'] = x.get('participantFrames').get('{}'.format(y)).get('championStats').get('magicPenPercent')
			frame['magicResist'] = x.get('participantFrames').get('{}'.format(y)).get('championStats').get('magicResist')
			frame['movementSpeed'] = x.get('participantFrames').get('{}'.format(y)).get('championStats').get('movementSpeed')
			frame['power'] = x.get('participantFrames').get('{}'.format(y)).get('championStats').get('power')
			frame['powerMax'] = x.get('participantFrames').get('{}'.format(y)).get('championStats').get('powerMax')
			frame['powerRegen'] = x.get('participantFrames').get('{}'.format(y)).get('championStats').get('powerRegen')
			frame['spellVamp'] = x.get('participantFrames').get('{}'.format(y)).get('championStats').get('spellVamp')
			frame['magicDamageDone'] = x.get('participantFrames').get('{}'.format(y)).get('damageStats').get('magicDamageDone')
			frame['magicDamageDoneToChampions'] = x.get('participantFrames').get('{}'.format(y)).get('damageStats').get('magicDamageDoneToChampions')
			frame['magicDamageTaken'] = x.get('participantFrames').get('{}'.format(y)).get('damageStats').get('magicDamageTaken')
			frame['physicalDamageDone'] = x.get('participantFrames').get('{}'.format(y)).get('damageStats').get('physicalDamageDone')
			frame['physicalDamageDoneToChampions'] = x.get('participantFrames').get('{}'.format(y)).get('damageStats').get('physicalDamageDoneToChampions')
			frame['physicalDamageTaken'] = x.get('participantFrames').get('{}'.format(y)).get('damageStats').get('physicalDamageTaken')
			frame['totalDamageDone'] = x.get('participantFrames').get('{}'.format(y)).get('damageStats').get('totalDamageDone')
			frame['totalDamageDoneToChampions'] = x.get('participantFrames').get('{}'.format(y)).get('damageStats').get('totalDamageDoneToChampions')
			frame['totalDamageTaken'] = x.get('participantFrames').get('{}'.format(y)).get('damageStats').get('totalDamageTaken')
			frame['trueDamageDone'] = x.get('participantFrames').get('{}'.format(y)).get('damageStats').get('trueDamageDone')
			frame['trueDamageDoneToChampions'] = x.get('participantFrames').get('{}'.format(y)).get('damageStats').get('trueDamageDoneToChampions')
			frame['trueDamageTaken'] = x.get('participantFrames').get('{}'.format(y)).get('damageStats').get('trueDamageTaken')
			
			frame['goldPerSecond'] = x.get('participantFrames').get('{}'.format(y)).get('jungleMinionsKilled')
			frame['jungleMinionsKilled'] = x.get('participantFrames').get('{}'.format(y)).get('jungleMinionsKilled')
			frame['level'] = x.get('participantFrames').get('{}'.format(y)).get('level')
			frame['minionsKilled'] = x.get('participantFrames').get('{}'.format(y)).get('minionsKilled')
			#frame['participantId'] = x.get('participantFrames').get('{}'.format(y)).get('participantId')
			frame['timeEnemySpentControlled'] = x.get('participantFrames').get('{}'.format(y)).get('timeEnemySpentControlled')
			frame['totalGold'] = x.get('participantFrames').get('{}'.format(y)).get('totalGold')
			frame['xp'] = x.get('participantFrames').get('{}'.format(y)).get('xp')

			print('Winner: {}'.format(winner))
			if winner == 100:
				if y in (1,2,3,4,5):
					frame['winner'] = 1
				else:
					frame['winner'] = 0
			elif winner == 200:
				if y in (1,2,3,4,5):
					frame['winner'] = 0
				else:
					frame['winner'] = 1
			print('Frame: {}'.format(json.dumps(frame)))
			all_frames.append(frame)
		
	return all_frames

In [45]:
db = OracleJSONDatabaseConnection()
print(db.get_collection_names())

json_object = list()

matches = db.open_collection('match_detail')

# Total documents:
print('Total documents: {}'.format(matches.find().count()))
# Sample train with just 1000. Objects are too big, and during development it's not worth it.
doc = matches.find().getOne()
content = doc.getContent()
#print(content)
built_object = build_final_object(content)
json_object.append(built_object)


C:\Users\nache/wallets/Wallet_eSportsDB
Connection successful.
['1v1_model', 'match', 'match_detail', 'matchups', 'predictor', 'summoner']
Total documents: 24232
100
Winner: 100
Frame: {"timestamp": 0, "abilityPower": 0, "armor": 36, "armorPen": 0, "armorPenPercent": 0, "attackDamage": 25, "attackSpeed": 100, "bonusArmorPenPercent": 0, "bonusMagicPenPercent": 0, "ccReduction": 0, "cooldownReduction": 0, "health": 575, "healthMax": 575, "healthRegen": 0, "lifesteal": 0, "magicPen": 0, "magicPenPercent": 0, "magicResist": 32, "movementSpeed": 345, "power": 200, "powerMax": 200, "powerRegen": 0, "spellVamp": 0, "magicDamageDone": 0, "magicDamageDoneToChampions": 0, "magicDamageTaken": 0, "physicalDamageDone": 0, "physicalDamageDoneToChampions": 0, "physicalDamageTaken": 0, "totalDamageDone": 0, "totalDamageDoneToChampions": 0, "totalDamageTaken": 0, "trueDamageDone": 0, "trueDamageDoneToChampions": 0, "trueDamageTaken": 0, "goldPerSecond": 0, "jungleMinionsKilled": 0, "level": 1, "minions

In [46]:
print('Inserted into JSON object with length {}'.format(len(json_object)))  # Total documents
  

Inserted into JSON object with length 1


In [47]:
#print(json_object)

[[{'timestamp': 0, 'abilityPower': 0, 'armor': 28, 'armorPen': 0, 'armorPenPercent': 0, 'attackDamage': 25, 'attackSpeed': 100, 'bonusArmorPenPercent': 0, 'bonusMagicPenPercent': 0, 'ccReduction': 0, 'cooldownReduction': 0, 'health': 500, 'healthMax': 500, 'healthRegen': 0, 'lifesteal': 0, 'magicPen': 0, 'magicPenPercent': 0, 'magicResist': 30, 'movementSpeed': 315, 'power': 350, 'powerMax': 350, 'powerRegen': 0, 'spellVamp': 0, 'magicDamageDone': 0, 'magicDamageDoneToChampions': 0, 'magicDamageTaken': 0, 'physicalDamageDone': 0, 'physicalDamageDoneToChampions': 0, 'physicalDamageTaken': 0, 'totalDamageDone': 0, 'totalDamageDoneToChampions': 0, 'totalDamageTaken': 0, 'trueDamageDone': 0, 'trueDamageDoneToChampions': 0, 'trueDamageTaken': 0, 'goldPerSecond': 0, 'jungleMinionsKilled': 0, 'level': 1, 'minionsKilled': 0, 'timeEnemySpentControlled': 0, 'totalGold': 500, 'xp': 0, 'winner': 0}, {'timestamp': 0, 'abilityPower': 0, 'armor': 28, 'armorPen': 0, 'armorPenPercent': 0, 'attackDamage

In [49]:
df = pd.read_json(json.dumps(json_object[0]), orient='records')

df.head(5)


,timestamp,abilityPower,armor,armorPen,armorPenPercent,attackDamage,attackSpeed,bonusArmorPenPercent,bonusMagicPenPercent,ccReduction,...,trueDamageDoneToChampions,trueDamageTaken,goldPerSecond,jungleMinionsKilled,level,minionsKilled,timeEnemySpentControlled,totalGold,xp,winner
0,0,0,28,0,0,25,100,0,0,0,...,0,0,0,0,1,0,0,500,0,0
1,0,0,28,0,0,25,100,0,0,0,...,0,0,0,0,1,0,0,500,0,0
2,0,0,28,0,0,25,100,0,0,0,...,0,0,0,0,1,0,0,500,0,0
3,0,0,28,0,0,25,100,0,0,0,...,0,0,0,0,1,0,0,500,0,0
4,0,0,28,0,0,25,100,0,0,0,...,0,0,0,0,1,0,0,500,0,0


In [32]:
df.head(5)

AttributeError: 'str' object has no attribute 'head'

In [ ]:
from pandas_profiling import ProfileReport

In [ ]:
df = pd.read_csv('../input/league-of-legends-1v1-matchups-results/matchups.csv')
report = ProfileReport(df)
report

In [ ]:
## install packages
!python3 -m pip install -q autogluon
!python3 -m pip install -q scikit-learn

In [ ]:
!python3 -m pip install -q autogluon


In [ ]:
from autogluon.tabular import TabularPredictor, TabularDataset

In [ ]:
df = TabularDataset('../input/league-of-legends-1v1-matchups-results/matchups.csv')

train = df.sample(frac=0.8,random_state=200) #random state is a seed value
test = df.drop(train.index)

train.head()

In [ ]:
label = 'WIN'

In [ ]:
save_path = './autogluon_trained_models'  # specifies folder to store trained models
predictor = TabularPredictor(label=label, path=save_path).fit(train)

In [ ]:
y_test = test[label]  # values to predict
test_data_nolabel = test.drop(columns=[label])  # delete label column to prove we're not cheating
test_data_nolabel.head()

In [ ]:
save_path = './autogluon_trained_models'  # specifies folder to store trained models

predictor = TabularPredictor.load(save_path)

y_pred = predictor.predict(test_data_nolabel)
print("Predictions:  \n", y_pred)
perf = predictor.evaluate_predictions(y_true=y_test, y_pred=y_pred, auxiliary_metrics=True)


In [ ]:
predictor.leaderboard(test, silent=True)